<a href="https://colab.research.google.com/github/mathijsvdv/FastBook.jl/blob/main/FastAI_jl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FastAI.jl for Colab

This is a template for running [FastAI.jl](https://github.com/FluxML/FastAI.jl) on Colab to get free access to a GPU. Since Julia is not an officially supported language, you will have to follow the steps in the *Setup* section the first time you run it and whenever the workspace is reset.

## Setup

1. Work on a copy of this notebook: File > Save a copy in Drive (you will need a Google account). Alternatively, you can download the notebook using File > Download .ipynb, then upload it to Colab.
2. If you need a GPU: Runtime > Change runtime type > Harware accelerator = GPU.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update JULIA_VERSION and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.


In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.2"
export JULIA_PACKAGES="CUDA IJulia CairoMakie"
JULIA_NUM_THREADS="2"
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  echo "Installing Julia packages, this may take up to 15 minutes. "
  julia -e 'using Pkg; Pkg.add(["CUDA", "IJulia", "CairoMakie"]); Pkg.add(Pkg.PackageSpec(url="https://github.com/FluxML/FastAI.jl")); Pkg.precompile()' &> /dev/null

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

Installing Julia 1.6.2 on the current Colab Runtime...
2021-08-27 12:51:24 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.2-linux-x86_64.tar.gz [112946671/112946671] -> "/tmp/julia.tar.gz" [1]
Installing Julia packages, this may take up to 15 minutes. 


## FastAI.jl

Here you can execute Julia code as usual, for example following [this tutorial](https://fluxml.ai/FastAI.jl/dev/notebooks/fitonecycle.ipynb.html). Note that the first time running training code in a session will take a long time for compilation. On subsequent runs, the code will not have to be recompiled as long as you keep the session alive.

In [ ]:
versioninfo()

Julia Version 1.6.2
Commit 1b93d53fc4 (2021-07-14 15:36 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, haswell)
Environment:
  JULIA_NUM_THREADS = 2


In [ ]:
using FastAI
data, blocks = loaddataset("imagenette2-160", (Image, Label))
method = ImageClassificationSingle(blocks)
learner = methodlearner(method, data, Models.xresnet18(), ToGPU(), Metrics(accuracy))
fitonecycle!(learner, 10, 0.002)

This program has requested access to the data dependency fastai-imagenette2-160.
which is not currently installed. It can be installed automatically, and you will not see this message again.

"imagenette2-160" from the fastai dataset repository (https://course.fast.ai/datasets)



Download size: ???



Do you want to download the dataset from https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-160.tgz to "/root/.julia/datadeps/fastai-imagenette2-160"?
[y/n]
stdin> y


┌ Info: Downloading
│   source = https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-160.tgz
│   dest = /root/.julia/datadeps/fastai-imagenette2-160/imagenette2-160.tgz
│   progress = 1.0
│   time_taken = 3.41 s
│   time_remaining = 0.0 s
│   average_speed = 27.721 MiB/s
│   downloaded = 94.417 MiB
│   remaining = 0 bytes
│   total = 94.417 MiB
└ @ HTTP /root/.julia/packages/HTTP/4AvE2/src/download.jl:128



7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)


Extracting archive: 
--
Path = 
Type = tar
Code Page = UTF-8

Everything is Ok

Folders: 23
Files: 13397
Size:       107794109
Compressed: 6872064


Epoch 1 TrainingPhase(): 100%|██████████████████████████| Time: 0:04:29


┌───────────────┬───────┬────────┬──────────┐
│         Phase │ Epoch │   Loss │ Accuracy │
├───────────────┼───────┼────────┼──────────┤
│ TrainingPhase │   1.0 │ 1.9144 │  0.39228 │
└───────────────┴───────┴────────┴──────────┘


...

┌─────────────────┬───────┬─────────┬──────────┐
│           Phase │ Epoch │    Loss │ Accuracy │
├─────────────────┼───────┼─────────┼──────────┤
│ ValidationPhase │   4.0 │ 0.70613 │   0.7779 │
└─────────────────┴───────┴─────────┴──────────┘


Epoch 5 TrainingPhase(): 100%|██████████████████████████| Time: 0:01:11


┌───────────────┬───────┬─────────┬──────────┐
│         Phase │ Epoch │    Loss │ Accuracy │
├───────────────┼───────┼─────────┼──────────┤
│ TrainingPhase │   5.0 │ 0.74564 │  0.75756 │
└───────────────┴───────┴─────────┴──────────┘


Epoch 5 ValidationPhase(): 100%|████████████████████████| Time: 0:00:07


┌─────────────────┬───────┬─────────┬──────────┐
│           Phase │ Epoch │    Loss │ Accuracy │
├─────────────────┼───────┼─────────┼──────────┤
│ ValidationPhase │   5.0 │ 0.64391 │  0.80417 │
└─────────────────┴───────┴─────────┴──────────┘


In [ ]:
import CairoMakie
plotpredictions(method, learner)